<div style="text-align: center; font-size: 30px; color: red; font-family: 'Arial', sans-serif;">
    <b>Predicción del Abandono de Clientes en Beta Bank usando Modelos de Aprendizaje Supervisado</b>
</div>

# Contenido <a id='back'></a>

* [Introducción](#intro)
* [Etapa 1. Carga y Exploración de los Datos](#data_review)
    * [1.1 Descripción de los Datos](#data_review)
* [Etapa 2. Preparación de los Datos](#data_preparing)
    * [2.1 División del conjunto de entrenamiento y de prueba](#data_divisions)
    * [2.2 Escalamiento de los datos](#data_scaler)
    * [2.3 Codificación One-Hot Encoding de las variables categóricas](#data_one-hot)
    * [Conclusiones de la Etapa 2](#conclusions_etapa2)
* [Etapa 3. Desarrollo del Modelo](#data_model)
    * [3.1 Equilibrio de Clases](#class_balance)
    * [3.2 Entrenamiento de los 4 modelos con desequilibrio de clases](#no_balance_class)
    * [3.3 Equilibrio de la clase minoritaria (oversampling](#train_oversampling)
    * [3.4 Equilibrio de la clase mayoritaria (undersampling)](#train_undersampling)
* [Etapa 4. Selección del mejor modelo](#model_evaluation)
    * [4.1 Modelo con mejores resultados en las métricas](#good_model)
* [Conclusiones finales del proyecto](#end)

## Introducción <a id='intro'></a>

En el competitivo mundo de la banca, retener a los clientes existentes es significativamente más económico que adquirir nuevos. Beta Bank ha identificado que un número creciente de sus clientes está abandonando el servicio mes tras mes. Con el objetivo de mitigar esta pérdida, el banco busca predecir cuáles de sus clientes están en riesgo de irse. Esto permitiría implementar estrategias de retención efectivas.

Este proyecto de Data Science se centrará en la creación de un modelo predictivo basado en el comportamiento histórico de los clientes. Utilizando técnicas de aprendizaje supervisado, se espera que el modelo pueda identificar con precisión los clientes propensos a abandonar el banco. El éxito del proyecto será evaluado con la métrica F1, con una meta mínima de 0.59, y se complementará con el análisis de la métrica AUC-ROC para evaluar el desempeño general del modelo.

El proyecto abordará varios desafíos clave, entre ellos el desequilibrio de clases, que será corregido mediante al menos dos enfoques diferentes para garantizar la calidad del modelo. Al finalizar, el modelo se probará en un conjunto de prueba para confirmar su eficacia en un entorno real.


# Etapa 1: Carga y exploración de datos <a id='data_review'></a>

In [76]:
# Cargamos los módulos a utilizar, e importamos funciones específicas de módulos
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler 
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.metrics import f1_score, roc_auc_score
from sklearn.utils import resample
import pprint

In [4]:
# Cargamos el conjunto de datos, el cual llamaremos `data`
data = pd.read_csv('/home/josue/Aprendizaje_supervisado/Churn.csv')

## Descripción de los datos <a id='data_review'></a>

In [18]:
# Revisamos a detalle el conjunto de datos

print('Observación de todo el conjunto de datos:', '\n', data.head())
print('\n')
print(f'Dimensiones del conjunto de datos:', data.shape)
print('\n')
print(data.info())
print('\n')
print('Conteo de personas que han abandonado el Banco:', data['Exited'].value_counts())

Observación de todo el conjunto de datos: 
    RowNumber  CustomerId   Surname  CreditScore Geography  Gender  Age  \
0          1    15634602  Hargrave          619    France  Female   42   
1          2    15647311      Hill          608     Spain  Female   41   
2          3    15619304      Onio          502    France  Female   42   
3          4    15701354      Boni          699    France  Female   39   
4          5    15737888  Mitchell          850     Spain  Female   43   

   Tenure    Balance  NumOfProducts  HasCrCard  IsActiveMember  \
0     2.0       0.00              1          1               1   
1     1.0   83807.86              1          0               1   
2     8.0  159660.80              3          1               0   
3     1.0       0.00              2          0               0   
4     2.0  125510.82              1          1               1   

   EstimatedSalary  Exited  
0        101348.88       1  
1        112542.58       0  
2        113931.57       1 

Después de analizar el conjunto de datos, vemos dos cosas que corregir.  
    1.  Normalizar los títulos de las observaciones.   
    2.  Tratar los valores faltantes en la observación `Tenure` mediante un modelo que es KNN Imputer y imputará los valores faltantes.  
    

In [20]:
# 1. Normalizar las Observaciones del conjunto de datos

data.columns = data.columns.str.lower()
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   rownumber        10000 non-null  int64  
 1   customerid       10000 non-null  int64  
 2   surname          10000 non-null  object 
 3   creditscore      10000 non-null  int64  
 4   geography        10000 non-null  object 
 5   gender           10000 non-null  object 
 6   age              10000 non-null  int64  
 7   tenure           9091 non-null   float64
 8   balance          10000 non-null  float64
 9   numofproducts    10000 non-null  int64  
 10  hascrcard        10000 non-null  int64  
 11  isactivemember   10000 non-null  int64  
 12  estimatedsalary  10000 non-null  float64
 13  exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


In [26]:
# 2. Imputar valores faltantes en `tenure`

# Creamos el Objeto KNNImputer
imputer = KNNImputer(n_neighbors=5) # Seleccionamos un total de 5 vecinos

# Seleccionamos las características relevantes para la imputación
columns = ['creditscore', 'age', 'balance', 'numofproducts', 'hascrcard', 'isactivemember', 'estimatedsalary', 'tenure']

# Aplicar KNN imputer
data[columns] = imputer.fit_transform(data[columns])

# Verificamos que no haya valores ausentes en `tenure`
print(data['tenure'].isnull().sum())

# Verificamos que no haya observaciones duplicadas
print(data.duplicated().sum())

0
0


Después de realizar estas correcciones al conjunto de datos, hemos visto que la calidad de los datos es mejor, ahora pasemos a la siguiente etapa que será donde nos encargaremos de la preparación de los datos, para después utilizar el modelo de Machine Learning.

# Etapa 2. Preparación de los datos <a id='data_preparing'></a>

## 2.1 División del conjunto de entrenamiento y de prueba <a id='data_divisions'></a>

In [31]:
# Dividimos nuestro conjunto de datos en `features` y `target`, cabe destacar que en `features` eliminaremos características que no son de mucha utilidad para el modelo.
features = data.drop(columns=['rownumber', 'customerid', 'surname', 'exited'], axis=1)
target = data['exited']

# Dividimos nuestro conjunto de datos en entrenamiento y prueba (80% para entrenamiento y 20% para prueba)
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.2, random_state=12345)

## 2.2 Escalamiento del conjunto de datos con Scaler <a id='data_scaler'></a>

In [33]:
# Para que un modelo funcione bien y sin priorizar cierto tipo de características, es importante escalarlo, con esto el modelo evaluará por igual todas las características

# Seleccionamos las características a escalar
numerical_columns_scale = ['creditscore', 'age', 'tenure', 'balance', 'numofproducts','estimatedsalary']

# Creamos un objeto Scaler
scaler = StandardScaler()

# Aplicación de scaler en nuestro conjunto de entrenamiento y prueba
features_train[numerical_columns_scale] = scaler.fit_transform(features_train[numerical_columns_scale])
features_test[numerical_columns_scale] = scaler.transform(features_test[numerical_columns_scale])

## 2.3 Codificación de variables categóricas (One-Hot Encoding) <a id='data_one-hot'></a>

In [34]:
# Hacemos la codificación One-Hot Encoding y convertimos las variables categóricas en binarias
features_train = pd.get_dummies(features_train, drop_first=True)
features_test = pd.get_dummies(features_test, drop_first=True)

# Utilizamos `align` para asegurarnos que tanto el conjunto de entrenamiento como de prueba tengan las mismas columnas
features_train, features_test = features_train.align(features_test, join='Left', axis=1, fill_value=0)

## Conclusiones de la Etapa 2 <a id='conclusions_etapa2'>

En esta sección de preparación de los datos, nos encargamos primero de segmentar los datos, en `features` y el `target` eliminando algunas columnas para `features`. Posteriormente dividimos nuestro conjunto de datos, con la ayuda de la función `train_test_split`, una vez haciendo esto, reservamos el 80% del conjunto de datos para el entrenamiento y el otro 20% para prueba. Debido a que nuestro conjunto hay variables numéricas con diferentes valores, era necesario escalaras con la media y una desviación estándar de 1, con `StandardScaler` realizamos esto y escalamos todas las variables numéricas.

Finalmente, asi como tuvimos que tratar de una forma las variables numéricas, las categóricas por su parte también lo necesita. Para las categóricas utilizaremos la codificación One-Hot Encoding, lo que ayudara a que las variables categóricas se conviertas a variables binarias, esto con el propósito, de que nuestro modelo de ML, trabaje sin ningún problema.

Para finalizar, con la ayuda de la función `align`, nos aseguramos que tanto el conjunto de entrenamiento tenga la misma dimensión de datos, para que el modelo de ML, trabaje sin ningún problema.

# Etapa 3: Desarrollo del Modelo <a id='data_model'></a>

## 3.1 Equilibrio de las clases <a id='class_balance'></a>

In [66]:
# Observación del equilibrio de nuestras clases
print(target_train.value_counts(normalize=True))

exited
0    0.79875
1    0.20125
Name: proportion, dtype: float64


Como vemos tenemos un desequilibrio de clases, puesto que para los clientes que no abandonaron el banco es de un 80%, pero de los que si un 20%, lo cual nos muestra el gran desequilibrio que tienen estas clases, de seguir así el modelo podría anteponer la mayoría de clases y sesgarse hacia esa clase y la minoría relegarla. Con esto el resultado sería que el modelo tendrá sesgo y no será preciso en sus predicciones. Ahora que tenemos listos los datos y la información de clase dominante, veamos lo que sigue. 

Primero para ver como el desequilibrio de clases sesga el resultado de modelo, el entrenamiento y las predicciones las haremos sin tratar este desequilibrio de clases, una vez que veamos los resultados seguiremos adelante con obtener los mismos resultados pero corrigiendo el desequilibrio de clases con la función `resample` de dos formas, una mediante el `oversampling` y otra mediante el `undersampling`

A continuación, entrenaremos nuestro conjunto de datos con 4 modelos, haremos una función que reciba los parámetros de entrenamiento y prueba, después iniciaremos los modelos y con la ayuda de un diccionario y dos ciclos obtendremos las métricas F1 Score y AUC-ROC.

## 3.2 Entrenamiento de los 4 modelos con desequilibrio de clases <a id='no_balance_class'></a>

In [52]:
# En esta sección vamos a utilizar 4 Modelos, de los cuales escogeremos el que de la mayor tasa de éxito, también, veremos que parámetros deben ser cambiados para ajustarlo lo máximo posible, los modelos de ML a utilizar son:
# 1. LogisticRegression
# 2. RandomForestClassifier
# 3. GradientBoostingClassifier
# 4. SVC

# Creamos una función llamada `train_and_evaluate_model` que se encargará de entrenar el modelo, hacer las predicciones y darnos el resultado de las métricas F1 y AUC-ROC.
def train_and_evaluate_model(model, features_train, features_test, target_train, target_test):
    # Entrenar el modelo
    model.fit(features_train, target_train)

    # Predecir con el modelo
    predictions = model.predict(features_test)

    # Calcula F1 Score
    f1 = f1_score(target_test, predictions)

    # Calcula AUC-ROC
    proba = model.predict_proba(features_test)[:, 1] if hasattr(model, 'predict_proba') else None
    auc_score = roc_auc_score(target_test, proba) if proba is not None else 'N/A'
    return f1, auc_score

In [53]:
# Inicializamos los 4 modelos de ML

logistic_reg = LogisticRegression(random_state=12345) # Modelo 1
random_forest = RandomForestClassifier(random_state=12345) # Modelo 2
svm_model = SVC(probability=True, random_state=12345) # Modelo 3, agregamos `probability=True` para poder calcular AUC-ROC
gradient_boosting = GradientBoostingClassifier(random_state=12345) # Modelo 4

# Lista de Modelos
models = {
    'Logistic Regression': logistic_reg,
    'Random Forest': random_forest,
    'SVM Model': svm_model,
    'Gradient Boosting': gradient_boosting
}

# Variable para almacenar los resultados
results = {}

# Entrenar y evaluar cada Modelo
for model_name, model in models.items():
    f1, auc_score = train_and_evaluate_model(model, features_train, features_test, target_train, target_test)
    results[model_name] = {'F1 Score': f1, 'AUC-ROC': auc_score}

# Mostrar los resultados
for model_name, result in results.items():
    print(f'Modelo: {model_name}')
    print(f"F1 Score: {result['F1 Score']}")
    print(f"AUC-ROC: {result['AUC-ROC']}")
    print('\n')


Modelo: Logistic Regression
F1 Score: 0.2857142857142857
AUC-ROC: 0.7576119856298695


Modelo: Random Forest
F1 Score: 0.564179104477612
AUC-ROC: 0.8608113198277133


Modelo: SVM Model
F1 Score: 0.49752883031301476
AUC-ROC: 0.8468640152693805


Modelo: Gradient Boosting
F1 Score: 0.5832106038291606
AUC-ROC: 0.8776394991000058




Después de probar los 4 Modelos de ML, observamos distintos resultados de F1 Score y AUC-ROC, ahora tenemos que ver el desequilibrio de las clases, ya que ocupamos un nivel más alto de F1 Score, con estos resultados, podemos darnos cuenta que el desequilibrio de las clases se está haciendo presente en los resultados de las métricas, por esta razón tenemos que equilibrarlas y entrenar nuevamente los modelos de ML.

## 3.3 Equilibrio de la clase minoritaria (oversampling) <a id='train_oversampling'></a>

In [62]:
# Incremento de la clase minoritaria con `resample` (oversampling) 

# Combinar `features_train` y `features_test` en un solo DF
train_data = pd.concat([features_train, target_train], axis=1)

# Separar las clases: clase mayoritaria (clientes que no se van) y clase minoritaria (clientes que se van)
class_majority = train_data[train_data['exited'] == 0] # Clientes que no se van
class_minority = train_data[train_data['exited'] == 1] # Clientes que se van

# Aumentar la clase minoritaria (ovsersampling) con `resample`
class_minority_ovsersampling = resample(class_minority, replace=True, n_samples= len(class_majority), random_state=12345)

# Combinar ambas clases mayoritaria y minoritaria
class_train = pd.concat([class_majority, class_minority_ovsersampling])

# Separar el conjunto de datos en train y test
features_train_oversampling = class_train.drop(columns='exited')
target_train_oversampling = class_train['exited']

## Evaluación del Modelo con equilibrio de clases (oversampling)

Como previamente realizamos una función para entrenar los 4 modelos propuestos para la resolución del problema, ahora la usaremos nuevamente, solo crearemos un nuevo diccionario que es donde guardaremos los resultados de evaluación del modelo con (oversampling)

In [63]:
# Variable para almacenar los resultados
results_oversampling = {}

# Entrenar y evaluar cada Modelo
for model_name, model in models.items():
    f1, auc_score = train_and_evaluate_model(model, features_train_oversampling, features_test, target_train_oversampling, target_test)
    results_oversampling[model_name] = {'F1 Score': f1, 'AUC-ROC': auc_score}

# Mostrar los resultados
for model_name, result in results_oversampling.items():
    print(f'Modelo: {model_name}')
    print(f"F1 Score: {result['F1 Score']}")
    print(f"AUC-ROC: {result['AUC-ROC']}")
    print('\n')

Modelo: Logistic Regression
F1 Score: 0.5016835016835017
AUC-ROC: 0.7628556242565184


Modelo: Random Forest
F1 Score: 0.5915875169606513
AUC-ROC: 0.8467888296502306


Modelo: SVM Model
F1 Score: 0.6115384615384616
AUC-ROC: 0.8580242410346732


Modelo: Gradient Boosting
F1 Score: 0.6302439024390243
AUC-ROC: 0.8756601074037735




Después de aplicar resample y realizar un oversampling, vemos que los resultados de las métricas suben en una cantidad considerable, claramente, el modelo está evaluando mejor, debido a que las clases están parejas, entonces nuestros modelos no se están sesgando hacia uno o otro lado, sin duda, el evaluar el desequilibrio de las clases es un punto muy importante a tomar en cuenta, cuando se resuelven problemas de ML.

## 3.4 Equilibrio de la clase mayoritaria (undersampling)<a id='train_undersampling'></a>

In [64]:
# Decremento con la clase mayoritaria con `resample` (unsersampling)

# Como anteriormente ya concatenamos los conjuntos de datos de entrenamiento y después obtuvimos la clase mayoritaria y minoritaria, utilizaremos esos mismas variables para no redundar en código.

# Decrementando el conjunto con la clase mayoritaria
class_majority_undersampling = resample(class_majority, replace=False, n_samples= len(class_minority), random_state=12345)

# Concatenación de la clase mayoritaria con la minoritaria del mismo tamaño
class_minority_train = pd.concat([class_minority, class_majority_undersampling])

# Separar el conjunto de datos en train y test
features_train_undersampling = class_minority_train.drop(columns='exited')
target_train_undersampling = class_minority_train['exited']

## Evaluación del Modelo con equilibrio de clases (undersampling)

In [65]:
# Variable para almacenar los resultados
results_undersampling = {}

# Entrenar y evaluar cada Modelo
for model_name, model in models.items():
    f1, auc_score = train_and_evaluate_model(model, features_train_undersampling, features_test, target_train_undersampling, target_test)
    results_undersampling[model_name] = {'F1 Score': f1, 'AUC-ROC': auc_score}

# Mostrar los resultados
for model_name, result in results_undersampling.items():
    print(f'Modelo: {model_name}')
    print(f"F1 Score: {result['F1 Score']}")
    print(f"AUC-ROC: {result['AUC-ROC']}")
    print('\n')

Modelo: Logistic Regression
F1 Score: 0.5016501650165017
AUC-ROC: 0.7597231382626315


Modelo: Random Forest
F1 Score: 0.6007194244604317
AUC-ROC: 0.8594468125019541


Modelo: SVM Model
F1 Score: 0.6145354185832567
AUC-ROC: 0.8651616639694136


Modelo: Gradient Boosting
F1 Score: 0.6232558139534883
AUC-ROC: 0.875363831399599




Después de utilizar la función resample y hacer un undersampling, nos damos cuenta que los resultados siguen siendo muy favorables en las métricas, lo cual nos deja ver que ocupar una u otra técnica es de gran ayuda, el undersampling la ventaja pueden ser sus tiempos de ejecución, en modelos con miles de datos o cientos de miles, hacer un oversampling ocupará muchos recursos y el tiempo será mayor, debido a que la clase minoritaria se le pasarán los mismos datos que la mayoritaria, men cambio con undersampling, puede que se ahorren mas recursos y tiempos debido, a que el conjunto de datos se reduce a la clase minoritaria, más sin embargo, aquí entra la evaluación de los Científicos de Datos, generalmente siempre se busca llegar al valor máximo de las métricas, entonces siempre debemos ver la forma, de obtener las mejores métricas para que el modelo sea lo más confiable posible.

# Etapa 4. Selección del mejor modelo <a id='model_evaluation'></a>

Después de analizar varios modelos, y de ajustas el desequilibrio de clases, tenemos que escoger cual es el mejor modelo que genere los mejores resultados en las métricas

##  4.1 Modelo con mejores resultados en las métricas<a id='good_model'></a>

In [75]:
# Selección del mejor modelo con sus mejores métricas de evaluación

# Concatenar los diccionarios con los resultados de las métricas de los modelos
results_final = {'Resultados con desequilibrio de clases': results,
                 'Resultados sin desequilibrio de clases (oversampling)': results_oversampling,
                 'Resultados sin desequilibrio de clases (undersampling)': results_undersampling}

# Inicializar variables para guardar el mejor modelo
best_f1_score = 0
best_auc_roc = 0
best_f1_model = ''
best_auc_roc_model = ''
best_f1_category = ''
best_auc_roc_category = ''

# Bucle para encontrar la mejores métricas en cada categoría y modelo
for category, models in results_final.items():
    for model, metric in models.items():
        # Comparar F1 Score
        if metric['F1 Score'] > best_f1_score:
            best_f1_score = metric['F1 Score']
            best_f1_model = model
            best_f1_category = category
        # Comparar AUC-ROC
        if metric['AUC-ROC'] > best_auc_roc:
            best_auc_roc = metric['AUC-ROC']
            best_auc_roc_model = model
            best_auc_roc_category = category

# Imprimir los mejores resultados
print(f"\nMejor F1 Score: {best_f1_score}, Modelo: {best_f1_model}, Categoría: {best_f1_category}")
print(f"\nMejor AUC-ROC: {best_auc_roc}, Modelo: {best_auc_roc_model}, Categoría: {best_auc_roc_category}")


Mejor F1 Score: 0.6302439024390243, Modelo: Gradient Boosting, Categoría: Resultados sin desequilibrio de clases (oversampling)

Mejor AUC-ROC: 0.8776394991000058, Modelo: Gradient Boosting, Categoría: Resultados con desequilibrio de clases


Es interesante ver los resultados de las métricas finales, el valor F1 Score sin duda sería cuando no existiera desequilibrio de clases, pero el mejor resultado de AUC-ROC fue con desequilibrio, lo cual es entendible debido a que AUC-ROC no se fija en precisión, sino solo en que tan bien el modelo esta asignando las probabilidades, en cambio F1 Score mide que tan bien el modelo esta detectando los verdaderos positivos y los falsos positivos, f1 score, sin duda es sensible a un desequilibrio de las clases.

# Conclusiones finales del proyecto <a id='conclusions'></a>

### En este proyecto, el objetivo fue desarrollar un modelo de Machine Learning para predecir si un cliente de Beta Bank abandonaría el banco en el futuro. Esto permitiría a la institución implementar estrategias de retención antes de perder a esos clientes.

#### 1. Preparación de los datos
Se llevó a cabo una serie de pasos para preparar los datos de manera efectiva:

- **Limpieza y preprocesamiento**: Se manejaron los datos faltantes y se codificaron las variables categóricas usando técnicas como **One-Hot Encoding**.
- **Escalado de características**: Las características numéricas se escalaron utilizando **StandardScaler** para garantizar que todas las características estuvieran en la misma escala.
- **Manejo del desequilibrio de clases**: El conjunto de datos estaba altamente desequilibrado, ya que había más clientes que no se iban del banco en comparación con los que se iban. Para abordar esto, se aplicaron dos técnicas:
  - **Sobremuestreo (Oversampling)** de la clase minoritaria.
  - **Submuestreo (Undersampling)** de la clase mayoritaria.

#### 2. Modelos de Machine Learning probados
Se probaron varios modelos para encontrar el que mejor predijera el abandono de clientes:

- **Regresión Logística**
- **Random Forest**
- **Support Vector Machine (SVM)**
- **Gradient Boosting**

De todos los modelos probados, el **Gradient Boosting** fue el que arrojó los mejores resultados. Se obtuvo un **F1 Score de 0.6302** y un **AUC-ROC de 0.8757** cuando se aplicó la técnica de sobremuestreo para corregir el desequilibrio de clases.

#### 3. Comparación de métricas
- **F1 Score**: Esta métrica mejoró notablemente cuando se equilibraron las clases mediante el **sobremuestreo**, ya que el modelo pudo prestar más atención a la clase minoritaria (clientes que se van). Esto es clave en conjuntos de datos desequilibrados, donde un modelo que solo prediga la clase mayoritaria no sería útil.

- **AUC-ROC**: El **AUC-ROC** fue consistente y alto en todas las pruebas, incluso en el conjunto de datos desequilibrado. Esto refleja que el modelo es capaz de distinguir bien entre clientes que se van y los que se quedan, basándose en las probabilidades asignadas a cada clase.

#### 4. Resultados
El modelo de **Gradient Boosting** no solo superó el valor mínimo de F1 Score requerido (**0.59**), sino que también mostró una excelente capacidad para distinguir entre clientes que se quedan y clientes que abandonan el banco, lo que sugiere que este modelo puede ser utilizado de manera efectiva en un entorno real.

#### 5. Conclusión general
El proyecto demostró que es posible construir un modelo efectivo para predecir el abandono de clientes utilizando técnicas avanzadas de Machine Learning y estrategias para manejar el desequilibrio de clases. El uso de **Gradient Boosting**, en particular, resultó ser una excelente opción por su capacidad para corregir los errores progresivamente y mejorar las predicciones.

A futuro, **Beta Bank** puede utilizar este modelo para identificar clientes en riesgo y tomar acciones proactivas para mejorar la retención, lo que será clave para reducir la pérdida de clientes y mejorar la rentabilidad.
